Intro to cuDF
=======================

Welcome to first cuDF tutorial notebook! This is a short introduction to cuDF, partly modeled after 10 Minutes to Pandas, geared primarily for new users. cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data. While we'll only cover some of the cuDF functionality, but at the end of this tutorial we hope you'll feel confident creating and analyzing GPU DataFrames.

We'll start by introducing the pandas library, and quickly move on cuDF.

In [1]:
import os
import numpy as np
import math
np.random.seed(12)

<a id="pandas"></a>
## Pandas

Data scientists typically work with two types of data: unstructured and structured. Unstructured data often comes in the form of text, images, or videos. Structured data - as the name suggests - comes in a structured form, often represented by a table or CSV. We'll focus the majority of these tutorials on working with structured data.

There exist many tools in the Python ecosystem for working with structured, tabular data but few are as widely used as Pandas. Pandas represents data in a table and allows a data scientist to manipulate the data to perform a number of useful operations such as filtering, transforming, aggregating, merging, visualizing and many more. 

For more information on Pandas, check out the excellent documentation: http://pandas.pydata.org/pandas-docs/stable/

Below we show how to create a Pandas DataFrame, an internal object for representing tabular data.

In [2]:
import pandas as pd; print('Pandas Version:', pd.__version__)


# here we create a Pandas DataFrame with
# two columns named "key" and "value"
df = pd.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

Pandas Version: 1.5.3
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


We can perform many operations on this data. For example, let's say we wanted to sum all values in the in the `value` column. We could accomplish this using the following syntax:

In [3]:
aggregation = df['value'].sum()
print(aggregation)

60.0


<a id="cudf"></a>
## cuDF

Pandas is fantastic for working with small datasets that fit into your system's memory and workflows that are not computationally intense. However, datasets are growing larger and data scientists are working with increasingly complex workloads - the need for accelerated computing is increasing rapidly.

cuDF is a package within the RAPIDS ecosystem that allows data scientists to easily migrate their existing Pandas workflows from CPU to GPU, where computations can leverage the immense parallelization that GPUs provide.

Below, we show how to create a cuDF DataFrame.

In [4]:
import cudf; print('cuDF Version:', cudf.__version__)


# here we create a cuDF DataFrame with
# two columns named "key" and "value"
df = cudf.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

cuDF Version: 23.10.02
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


As before, we can take this cuDF DataFrame and perform a `sum` operation over the `value` column. The key difference is that any operations we perform using cuDF use the GPU instead of the CPU.

In [5]:
aggregation = df['value'].sum()
print(aggregation)

60.0


Note how the syntax for both creating and manipulating a cuDF DataFrame is identical to the syntax necessary to create and manipulate Pandas DataFrames; the cuDF API is based on the Pandas API. This design choice minimizes the cognitive burden of switching from a CPU based workflow to a GPU based workflow and allows data scientists to focus on solving problems while benefitting from the speed of a GPU!

# DataFrame Basics with cuDF

In the following tutorial, you'll get a chance to familiarize yourself with cuDF. For those of you with experience using pandas, this should look nearly identical.

Along the way you'll notice small exercises. These exercises are designed to help you get a feel for writing the code yourself, but if you get stuck, you can take a look at the solutions.

Portions of this were borrowed from the 10 Minutes to cuDF guide.

Object Creation
---------------

Creating a `cudf.Series`.

In [6]:
s = cudf.Series([1,2,3,None,4])
print(s)

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64


Creating a `cudf.DataFrame` by specifying values for each column.

In [7]:
df = cudf.DataFrame([('a', list(range(20))),
('b', list(reversed(range(20)))),
('c', list(range(20)))])
print(df)

   0                                                  1
0  a  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1  b  [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8,...
2  c  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...


Creating a `cudf.DataFrame` from a `pd.Dataframe`.

In [8]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

   a     b
0  0   0.1
1  1   0.2
2  2  <NA>
3  3   0.3


Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [9]:
print(df.head(2))

   0                                                  1
0  a  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1  b  [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8,...


Sorting by values.

In [10]:
print(pdf.sort_values(by='b'))

   a    b
0  0  0.1
1  1  0.2
3  3  0.3
2  2  NaN


Selection
------------

## Getting

Selecting a single column, which initially yields a `cudf.Series` (equivalent to `df.a`).

In [11]:
print(pdf['a'])

0    0
1    1
2    2
3    3
Name: a, dtype: int64


## Selection by Label

Selecting rows from index 2 to index 5 from columns `a` and `b`.

In [12]:
print(pdf.loc[2:5, ['a', 'b']])

   a    b
2  2  NaN
3  3  0.3


## Selection by Position

Selecting via integers and integer slices, like numpy/pandas.

In [13]:
print(df.iloc[0])

   0                                                  1
0  a  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...


In [14]:
print(df.iloc[0:3, 0:2])

   0                                                  1
0  a  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1  b  [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8,...
2  c  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...


You can also select elements of a `DataFrame` or `Series` with direct index access.

In [15]:
print(df[3:5])

Empty DataFrame
Columns: [0, 1]
Index: []


In [16]:
print(s[3:5])

3    <NA>
4       4
dtype: int64


## Exercise 1

Try to select only the rows at index `4` and `9` from `df`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.iloc[[4,9]])
   </pre>
</details>

In [17]:
# Re-create DF for below examples.

In [82]:
df = cudf.DataFrame()

df['a'] = [float(i) for i in range(25)]
df['b'] = [float((i - 24) * -1) for i in range(25)]
df['c'] = [float(i+1) for i in range(25)]

df.head()

,a,b,c
0,0.0,24.0,1.0
1,1.0,23.0,2.0
2,2.0,22.0,3.0
3,3.0,21.0,4.0
4,4.0,20.0,5.0


In [53]:
print(df.iloc[[4,9]])

     a     b     c
4  4.0  20.0   5.0
9  9.0  15.0  10.0


## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [ ]:
print(df[df.b > 15])

Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [ ]:
print(df.query("b == 3"))  

In [ ]:
val = 3
df.query("b == @val")

You can also pass local variables to cuDF queries, via the `local_dict` keyword or `@` operator.

In [ ]:
cudf_comparator = 3
print(df.query("b == @cudf_comparator"))

Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

## Exercise 2

Try to select only the rows from `df` where the value in column `b` is greater than the value in column `c` + 6.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

In [ ]:
print(df.query("b > c + 6").sort_values(by='b', ascending=True))

Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [ ]:
print(s.fillna(999))

Operations
------------

## Stats

Calculating descriptive statistics for a `Series`.

In [ ]:
s = cudf.Series(np.arange(10)).astype(np.float32)

In [ ]:
print(s.mean(), s.var())

In [ ]:
print(df.describe())

## Applymap

Applying functions to a `Series`.

In [21]:
def add_ten(num):
    return num + 10

print(s.apply(add_ten))

0      11
1      12
2      13
3    <NA>
4      14
dtype: int64


We can also apply more complicated functions, such as this one.

In [22]:
def complex_math_transform(num):
    return math.cos(num) * 3 / 9

print(s.apply(complex_math_transform))

0     0.180100769
1    -0.138715612
2    -0.329997499
3            <NA>
4    -0.217881207
dtype: float64


## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the cuDF and [nvStrings API](https://docs.rapids.ai/api/nvstrings/nightly/) documentation for more information.

In [23]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
print(s.str.lower())

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: object


## Exercise 3

Try to convert all the strings to uppercase. Take a look at the nvStrings API documentation linked above if you need some help.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(s.str.upper())
   </pre>
</details>

In [24]:
print(s.str.upper())

0       A
1       B
2       C
3    AABA
4    BACA
5    <NA>
6    CABA
7     DOG
8     CAT
dtype: object


## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [25]:
s = cudf.Series([1, 2, 3, None, 5])
print(cudf.concat([s, s]))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64


## Append

Appending values from another `Series` or array-like object.

In [26]:
print(s.append(s))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64


/home/takasur/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/cudf/core/indexed_frame.py:3727: FutureWarning: The append method is deprecated and will be removed in a future version. Use cudf.concat instead.
  warnings.warn(


## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [27]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
print(merged)

  key  vals_a vals_b
0   a    10.0  100.0
1   c    12.0  101.0
2   e    14.0  102.0
3   b    11.0   <NA>
4   d    13.0   <NA>


## Exercise 4

Using the DataFrames we created above, try to do an `inner` join using `merge`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df_a.merge(df_b, on=['key'], how='inner'))
   </pre>
</details>

In [33]:
print(df_a.merge(df_b, on='key', how='inner'))

  key  vals_a  vals_b
0   a    10.0   100.0
1   c    12.0   101.0
2   e    14.0   102.0


## Grouping

Like pandas, cuDF supports the [Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) groupby paradigm.

In [54]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]
df.head()

,a,b,c,agg_col1,agg_col2
0,0.0,24.0,1.0,1,1
1,1.0,23.0,2.0,0,0
2,2.0,22.0,3.0,1,0
3,3.0,21.0,4.0,0,1
4,4.0,20.0,5.0,1,0


Grouping and then applying the `sum` function to the grouped data.

In [55]:
print(df.groupby('agg_col1').sum())

              a      b      c  agg_col2
agg_col1                               
1         156.0  156.0  169.0         5
0         144.0  144.0  156.0         4


Grouping and applying statistical functions to specific columns, using `agg`.

In [56]:
print(df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}))

             a     b      c
agg_col1                   
1         24.0  12.0  169.0
0         23.0  12.0  156.0


## Exercise 5

We can also group by multiple columns at once, which we call grouping hierarchically. Try to group `df` by `agg_col1` and `agg_col2` and calculate the mean of column `a` and minimum of column `b`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}).to_pandas())
   </pre>
</details>

In [57]:
print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}).to_pandas())

                      a    b
agg_col1 agg_col2           
0        0         12.0  1.0
1        0         12.0  2.0
         1         12.0  0.0
0        1         12.0  3.0


Time Series
------------


`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [60]:
date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = np.random.sample(len(date_df))
print(date_df.head())

        date     value
0 2018-11-20  0.686441
1 2018-11-21  0.265130
2 2018-11-22  0.397033
3 2018-11-23  0.513016
4 2018-11-24  0.344204


## Exercise 6

Try to use `query` to filter `date_df` to only those row with a date before `2018-11-23`. This is a bit trickier than the prior exercises. As a hint, you'll want to explore the `to_datetime` function from the `pandas` library.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>
    search_date = pd.to_datetime('2018-11-23')
    print(date_df.query("date &lt= @search_date"))
            </br>
   </pre>
</details>

In [64]:
search_date = pd.to_datetime('2018-11-23')
print(date_df.query("date <= @search_date"))

        date     value
0 2018-11-20  0.686441
1 2018-11-21  0.265130
2 2018-11-22  0.397033
3 2018-11-23  0.513016


You can also interact with datetime columns to extract things like the day, hour, minute, and more.

In [65]:
date_df['minute'] = date_df.date.dt.minute
print(date_df.head())

        date     value  minute
0 2018-11-20  0.686441       0
1 2018-11-21  0.265130       0
2 2018-11-22  0.397033       0
3 2018-11-23  0.513016       0
4 2018-11-24  0.344204       0


Converting Data Representation
--------------------------------

## Pandas

Converting a cuDF `DataFrame` to a pandas `DataFrame`.

In [66]:
print(df.head().to_pandas())

     a     b    c  agg_col1  agg_col2
0  0.0  24.0  1.0         1         1
1  1.0  23.0  2.0         0         0
2  2.0  22.0  3.0         1         0
3  3.0  21.0  4.0         0         1
4  4.0  20.0  5.0         1         0


## Numpy

Converting a cuDF `DataFrame` to a numpy `ndarray`.

In [69]:
# It looks like you are trying to use the as_matrix() method, which was deprecated in pandas and later removed. The error you're seeing indicates that the as_matrix() method is not available in cuDF either.

# Instead, you can use the to_numpy() method in both pandas and cuDF to get the underlying data as a NumPy array. Here's an example using cuDF:

print(df.to_numpy())

[[ 0. 24.  1.  1.  1.]
 [ 1. 23.  2.  0.  0.]
 [ 2. 22.  3.  1.  0.]
 [ 3. 21.  4.  0.  1.]
 [ 4. 20.  5.  1.  0.]
 [ 5. 19.  6.  0.  0.]
 [ 6. 18.  7.  1.  1.]
 [ 7. 17.  8.  0.  0.]
 [ 8. 16.  9.  1.  0.]
 [ 9. 15. 10.  0.  1.]
 [10. 14. 11.  1.  0.]
 [11. 13. 12.  0.  0.]
 [12. 12. 13.  1.  1.]
 [13. 11. 14.  0.  0.]
 [14. 10. 15.  1.  0.]
 [15.  9. 16.  0.  1.]
 [16.  8. 17.  1.  0.]
 [17.  7. 18.  0.  0.]
 [18.  6. 19.  1.  1.]
 [19.  5. 20.  0.  0.]
 [20.  4. 21.  1.  0.]
 [21.  3. 22.  0.  1.]
 [22.  2. 23.  1.  0.]
 [23.  1. 24.  0.  0.]
 [24.  0. 25.  1.  1.]]


Converting a cuDF `Series` to a numpy `ndarray`.

In [87]:
df.head()

,a,b,c
0,0.0,24.0,1.0
1,1.0,23.0,2.0
2,2.0,22.0,3.0
3,3.0,21.0,4.0
4,4.0,20.0,5.0


In [85]:
df['a'].values

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.])

Getting Data In/Out
------------------------


## CSV

Writing to a CSV file, by first sending data to a pandas `Dataframe` on the host.

In [88]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_pandas().to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [89]:
df = cudf.read_csv('example_output/foo.csv')
print(df)

       a     b     c
0    0.0  24.0   1.0
1    1.0  23.0   2.0
2    2.0  22.0   3.0
3    3.0  21.0   4.0
4    4.0  20.0   5.0
5    5.0  19.0   6.0
6    6.0  18.0   7.0
7    7.0  17.0   8.0
8    8.0  16.0   9.0
9    9.0  15.0  10.0
10  10.0  14.0  11.0
11  11.0  13.0  12.0
12  12.0  12.0  13.0
13  13.0  11.0  14.0
14  14.0  10.0  15.0
15  15.0   9.0  16.0
16  16.0   8.0  17.0
17  17.0   7.0  18.0
18  18.0   6.0  19.0
19  19.0   5.0  20.0
20  20.0   4.0  21.0
21  21.0   3.0  22.0
22  22.0   2.0  23.0
23  23.0   1.0  24.0
24  24.0   0.0  25.0


That's it! You've got the basics of cuDF down! Let's talk a little bit about the computational performance of cuDF and GPUs.

# Performace

One of the primary reasons to use cuDF over pandas is performance. For some workflows, the GPU can be **much** faster than the CPU. Let's illustrate this by starting with a small example: creating a DataFrame and calculating the sum of a column.

In [90]:
a = np.random.rand(10000000) # 10 million values

In [91]:
pdf = pd.DataFrame()
cdf = cudf.DataFrame()

In [92]:
%time pdf['a'] = a
%time cdf['a'] = a

CPU times: user 295 ms, sys: 448 ms, total: 743 ms
Wall time: 224 ms
CPU times: user 41.8 ms, sys: 103 ms, total: 145 ms
Wall time: 42.8 ms


In [93]:
%%timeit
pdf['a'].sum()

7.14 ms ± 71.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [94]:
%%timeit
cdf['a'].sum()

176 µs ± 473 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Pretty cool! This is a pretty small example, though.

### A More Realistic Example: Sensor Data Analytics

To get a more realistic sense of how powerful cuDF and GPUs can be, let's imagine you had a fleet of sensors that collect data every millisecond. These sensors could be measuring pressure, temperature, or something else entirely.

Let's imagine we want to analyze one day's worth of sensor data. We'll assign random values for the sensor `value` to use for this example. We'll start by creating the data, and generating some datetime related features like we learned about in the above tutorial.

In [95]:
%%time

date_df = pd.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute

print(date_df.shape)
date_df.head()

(86400001, 4)
CPU times: user 7.68 s, sys: 8.83 s, total: 16.5 s
Wall time: 4.95 s


,date,value,hour,minute
0,2019-07-05 00:00:00.000,0.925728,0,0
1,2019-07-05 00:00:00.001,0.444162,0,0
2,2019-07-05 00:00:00.002,0.480344,0,0
3,2019-07-05 00:00:00.003,0.626711,0,0
4,2019-07-05 00:00:00.004,0.616895,0,0


Just creating the data takes a while! Let's do our analysis. From our sensor data, we want to get the maximum sensor value for each minute. Since we don't want to combine values in the same minute of different hours, we'll need to do a hierarchical groupby.

In [96]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

CPU times: user 2.52 s, sys: 2.9 s, total: 5.42 s
Wall time: 1.61 s


value
hour minute          
0    0       0.999994
     1       0.999931
     2       0.999995
     3       0.999962
     4       0.999975

This is fairly slow! Imagine if we had a fleet of sensors. Time would become a serious concern.

Let's try this in cuDF now, using the GPU DataFrame. We'll run the same code as above.

In [97]:
%%time

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
date_df['second'] = date_df.date.dt.second

print(date_df.shape)
print(date_df.head())

(86400001, 5)
                     date     value  hour  minute  second
0 2019-07-05 00:00:00.000  0.103534     0       0       0
1 2019-07-05 00:00:00.001  0.611112     0       0       0
2 2019-07-05 00:00:00.002  0.969298     0       0       0
3 2019-07-05 00:00:00.003  0.054181     0       0       0
4 2019-07-05 00:00:00.004  0.389426     0       0       0
CPU times: user 1.02 s, sys: 1.62 s, total: 2.64 s
Wall time: 789 ms


In [98]:
%time results = date_df.groupby(['hour', 'minute', 'second']).agg({'value':'max'})
results.head()

CPU times: user 61.5 ms, sys: 98.4 ms, total: 160 ms
Wall time: 47.9 ms


,,,value
hour,minute,second,
2,43,35,0.999533
4,13,8,0.997685
1,44,5,0.999127
3,31,6,0.998985
9,49,13,0.998355


While the results may vary slightly, it should be clear that GPU acceleration can make a significant difference. We can get much faster results with the same code!

## Exercise 7

Play around with some more pandas and cuDF operations and compare the performance between them. What operations can you find that gives the highest performance ratio? 

You can start a cell with `%%time` to time the cell, or with `%%timeit`, which runs the cell multiple times and gives an average. `%%timeit` gives a more accurate benchmark but takes longer to run.

In [2]:
import pandas as pd
import cudf

In [3]:
%%time
ppdff = pd.read_csv('NW_data.csv')

CPU times: user 18.6 s, sys: 6.6 s, total: 25.2 s
Wall time: 25.2 s


In [4]:
%%time
cdf = cudf.read_csv('NW_data.csv')

CPU times: user 4.32 s, sys: 5.94 s, total: 10.3 s
Wall time: 3.69 s
